# Naive Machine Translation

This project implements a naive English→French translation model using word embeddings.
It includes both a nearest-neighbor baseline and an aligned embedding model using Procrustes optimization.
The goal is to explore vector-space semantics and evaluate how well word-level translation can work without large neural models.

### Imports

In [1]:
import numpy as np
import pickle #  used for serializing and deserializing Python object structures
import pandas as pd

## 1. Load Resources (Embeddings & Seed Dictionary)


In [ ]:
en_embeddings_subset = pickle.load(open("./data/en_embeddings.p", "rb")) # {word : array([])}
fr_embeddings_subset = pickle.load(open("./data/fr_embeddings.p", "rb"))

In [ ]:
def get_dict(file_name):
    """
    This function returns the english to french dictionary given a file where each column corresponds to a word.
    Check out the files this function takes in your workspace.
    
    """
    # Read the CSV file into a pandas DataFrame using a space as the delimiter  
    my_file = pd.read_csv(file_name, delimiter=' ')  

    # Initialize an empty dictionary to store the English to French translations  
    etof = {}  

    # Loop through each row in the DataFrame  
    for i in range(len(my_file)):  
        # Access the English word from the first column (index 0) of the current row  
        en = my_file.loc[i][0]  
        
        # Access the French translation from the second column (index 1) 
        # of the current row  
        fr = my_file.loc[i][1]  
        
        # Add the English word and its French translation to the dictionary  
        etof[en] = fr  

    # Return the populated dictionary containing English to French translations  
    return etof

In [ ]:
# loading the english to french dictionaries

en_fr_train = get_dict('./data/en-fr.train.txt') # key: English word, value: French word
print('The length of the English to French training dictionary is', len(en_fr_train))

en_fr_test = get_dict('./data/en-fr.test.txt') # {english:french}
print('The length of the English to French test dictionary is', len(en_fr_test))

The length of the English to French training dictionary is 5000
The length of the English to French test dictionary is 1500


## 2. Preprocessing & Utilities

In [12]:
def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''
    # you have to set this variable to the true label.
    cos = -10    
    dot = np.dot(A, B)
    normb = np.linalg.norm(B)
    
    if len(A.shape) == 1: # If A is just a vector, we get the norm
        norma = np.linalg.norm(A)
        cos = dot / (norma * normb)
    else: # If A is a matrix, 
        # then compute the norms of the word vectors of the matrix (norm of each row)
        norma = np.linalg.norm(A, axis=1)
        epsilon = 1.0e-9 # to avoid division by 0
        cos = dot / (norma * normb + epsilon)
        
    return cos

## 3. Nearest-Neighbor Baseline (Word→Word)

In [13]:
def nearest_neighbor(v, candidates, k=1, cosine_similarity=cosine_similarity):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    similarity_l = []
    

    # for each candidate vector...
    for row in candidates:
        # get the cosine similarity
        cos_similarity = cosine_similarity(row, v)

        # append the similarity to the list
        similarity_l.append(cos_similarity)

    
    # sort the similarity list and get the indices of the sorted list (ascending)    
    sorted_ids = np.argsort(similarity_l)
    
    # Reverse the order of the sorted_ids array (descending)
    sorted_ids = sorted_ids [::-1]
    
    
    # get the indices of the k most similar candidate vectors
    k_idx = sorted_ids[:k]
  
    return k_idx

## 4. Embedding Alignment (EN→FR) — Procrustes

In [6]:
def get_matrices(en_fr, french_vecs, english_vecs):
    """
    Creates matrices of word embeddings for English and 
    French words that are mapped to each other.
    
    Inputs:
        en_fr: Dictionary mapping English words to French words.
        french_vecs: Dictionary of French word embeddings.
        english_vecs: Dictionary of English word embeddings.
    
    Outputs: 
        X: Matrix with each row being the embedding of an English word. 
        Shape is (number_of_words, embedding_size).
        
        Y: Matrix with each row being the embedding of the corresponding French word. 
        Shape matches X.

    """
    # X_l and Y_l are lists of the english and french word embeddings
    X_l = list()
    Y_l = list()

    # getting the english words (the keys in the dictionary) and storing them in a set()
    english_set = set(english_vecs.keys())

    # getting the french words (keys in the dictionary) and storing in a set()
    french_set = set(french_vecs.keys())

    # loop through all english, french word pairs in the english french dictionary
    for en_word, fr_word in en_fr.items():

        # check that the french word has an embedding 
        # and that the english word has an embedding
        if fr_word in french_set and en_word in english_set:

            # get the english embedding
            en_vec = english_vecs[en_word]

            # get the french embedding
            fr_vec = french_vecs[fr_word]

            # add the english embedding to the list
            X_l.append(en_vec)

            # add the french embedding to the list
            Y_l.append(fr_vec)

    # stack the vectors of X_l into a matrix X
    X = np.vstack(X_l) # stack arrays in sequence vertically (row wise)

    # stack the vectors of Y_l into a matrix Y
    Y = np.vstack(Y_l)

    return X, Y

In [8]:
def compute_loss(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        L: the value of the loss function for given X, Y and R.
    '''
    # m is the number of rows in X -> number of samples
    m = X.shape[0]
        
    # diff is XR - Y    
    diff = np.dot(X, R) - Y

    # diff_squared is the element-wise square of the difference    
    diff_squared = np.square(diff)

    # sum_diff_squared is the sum of the squared elements
    sum_diff_squared = np.sum(diff_squared)

    # loss i is the sum_diff_squared divided by the number of examples (m)
    loss = sum_diff_squared / m
   
    return loss

In [9]:
def compute_gradient(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from 
        English to French vector space embeddings.
    Outputs:
        g: a scalar value - gradient of the loss function L for given X, Y and R.
    '''
    # m is the number of rows in X => number of samples
    m = X.shape[0]

    # gradient is X^T(XR - Y) * 2/m    
    gradient = np.dot(X.T, np.dot(X,R) - Y) * (2/m)
    
    return gradient

##### Finding the optimal R with Gradient Descent Algorithm

In [10]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003, verbose=True, compute_loss=compute_loss, compute_gradient=compute_gradient):
    '''
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    '''
    np.random.seed(129)

    # the number of columns in X is the number of dimensions for a word vector (e.g. 300)
    # R is a square matrix with length equal to the number of dimensions in th  word embedding
    
    # At first R is initialized with random numbers
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if verbose and i % 25 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
    
        # use the function that you defined to compute the gradient
        gradient = compute_gradient(X, Y, R)

        # update R by subtracting the learning rate times gradient
        R -= learning_rate*gradient # R = R - alpha*g
        
    return R

In [18]:
def translate_word(english_word, en_embeddings, fr_embeddings, translation_matrix):
    """
    Translate an English word to French using embeddings and a trained translation matrix.
    
    Parameters:
    english_word (str): The word in English to translate.
    en_embeddings (dict): English word embeddings (word -> vector).
    fr_embeddings (dict): French word embeddings (word -> vector).
    translation_matrix (np.ndarray): The trained translation matrix (from English to French).
    
    Returns:
    str: The translated French word.
    """
    # Step 1: Get the embedding for the English word
    if english_word in en_embeddings:
        en_word_embedding = en_embeddings[english_word]
    else:
        return f"Word '{english_word}' not found in the English embeddings."

    # Step 2: Apply the translation matrix to get the French embedding
    translated_embedding = np.dot(en_word_embedding, translation_matrix)

    # Step 3: Find the closest French word in the embedding space
    min_distance = float('inf')
    french_word = None

    for fr_word, fr_embedding in fr_embeddings.items():
        distance = np.linalg.norm(translated_embedding - fr_embedding)
        if distance < min_distance:
            min_distance = distance
            french_word = fr_word

    return french_word

In [ ]:
# getting the training set:
X_train, Y_train = get_matrices(en_fr_train, fr_embeddings_subset, 
                                en_embeddings_subset)

#### Calculate Transformation matrix R


In [11]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8)

loss at iteration 0 is: 963.0146
loss at iteration 25 is: 97.8292
loss at iteration 50 is: 26.8329
loss at iteration 75 is: 9.7893
loss at iteration 100 is: 4.3776
loss at iteration 125 is: 2.3281
loss at iteration 150 is: 1.4480
loss at iteration 175 is: 1.0338
loss at iteration 200 is: 0.8251
loss at iteration 225 is: 0.7145
loss at iteration 250 is: 0.6534
loss at iteration 275 is: 0.6185
loss at iteration 300 is: 0.5981
loss at iteration 325 is: 0.5858
loss at iteration 350 is: 0.5782
loss at iteration 375 is: 0.5735


In [ ]:
english_w = input("Enter the English word: ")
french_w = translate_word(english_w, en_embeddings_subset, fr_embeddings_subset, R_train)
print('English word:', english_w) 
print('French word:', french_w)

## 5. Evaluation (Accuracy / Top-k / Examples)

In [14]:
def test_vocabulary(X, Y, R, nearest_neighbor=nearest_neighbor):
    '''
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    '''

    # The prediction is X times R
    pred = np.dot(X,R)

    # initialize the number correct to zero
    num_correct = 0

    # loop through each row in pred (each transformed embedding)
    for i in range(len(pred)):
        # get the index of the nearest neighbor of pred at row 'i';
        # also pass in the candidates in Y
        pred_idx = nearest_neighbor(pred[i], Y, 1)

        # if the index of the nearest neighbor equals the row of i... \
        if pred_idx == i:
            # increment the number correct by 1.
            num_correct += 1

    # accuracy is the number correct divided by the number of rows in 'pred' 
    # (also number of rows in X)
    accuracy = num_correct / X.shape[0]


    return accuracy

Let's see how is your translation mechanism working on the unseen data:

In [15]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [16]:
acc = test_vocabulary(X_val, Y_val, R_train)  
print(f"accuracy on test set is {acc:.3f}")

accuracy on test set is 0.557


## 6. Sentence Stub (Optional for later)

In [ ]:
# Code: very simple word-by-word sentence translation; note limitations

## 7. sacreBLEU Setup (Optional) 

In [ ]:
# Code: install/use sacrebleu; compute BLEU on a tiny sentence set (later)

## 8. Error Analysis & Observations

## 9. Roadmap / Next Steps

In [17]:
# Text: what you'll add next (transformer baseline, UI, subwords, etc.)